# Cloud ETL - Amazon Pet Products Customer Reviews Dataset

In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install --only-upgrade openjdk-11-jdk-headless -qq > /dev/null
!wget -q -c http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [92.1 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.laun

In [2]:
!wget -c https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


--2022-09-19 23:09:56--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.00MB/s    in 0.2s    

2022-09-19 23:09:56 (5.00 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("bigdata-homework1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
pet_products_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
pet_products_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...|2015-08-31 00:00:00|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...| 

# Count the number or records (rows) in the dataset

In [5]:
# Count the number of records (rows) in the dataset
print(pet_products_df.count())
pet_products_df = pet_products_df.dropna()
print(pet_products_df.count())
pet_products_df = pet_products_df.dropDuplicates()
print(pet_products_df.count())



2643619
2643241
2643241


# Transform dataset to fit the tables in the schema file

In [6]:
pet_products_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



### Review_id DataFrame

In [7]:
# create the review_id dataframe for the 
from pyspark.sql.functions import to_date
# Review DataFrame
review_id_df = pet_products_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1015HJHGZUJVG|   26879429|B00076HSUC|     421502374| 2013-08-11|
|R1023YMO9XR93S|   49345442|B00171IVHA|      25146499| 2014-12-02|
|R10267J8PBWEVQ|   48576217|B0024E0QHC|     857059897| 2014-06-25|
|R103326H6Z3CTG|   34624253|B000UZNLGA|     144491764| 2011-11-12|
| R1035J76P280Y|   48425022|B003IRJTCC|     996646406| 2010-12-22|
| R103IP0FCBH8I|   14356933|B0002AQPKW|     152333311| 2011-12-12|
|R103WDQ4IDVIUV|   14302088|B00E8LYJS6|     394306248| 2014-12-15|
|R103WQ2GZQLH6T|   13853268|B006MUQO0M|      84171195| 2015-06-26|
|R104K30IR1GWUD|   27083213|B008V68GKG|     915709880| 2013-07-18|
|R104LLIZQ8O3DW|   18411324|B00ETS9LZ4|     259591253| 2014-06-05|
|R104OCMU5PF85T|    2654230|B008CO4H8C|     648340511| 2015-03-03|
|R104QASUJNTTNP|   21635417|B0056J2PWG|      26863086| 2014-04

### Products DataFrame

In [8]:
# Products DataFrame
products_df = pet_products_df.select(["product_id", "product_title"])
products_df = products_df.dropDuplicates()
products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00134HSYS|Special Edition P...|
|B00BS6NPBG|High Tech Pet 6-P...|
|B000F930FS|PetSafe ScoopFree...|
|B000FJ9QTW|Insight ActiviToy...|
|B000ALY0OQ|SmartCat Bootsie'...|
|B00Q8ETIZ0|Dogloveit Rubber ...|
|B002VU2BA4|Kragonfly Interch...|
|B00QA3K3QM|Attmu Retractable...|
|B005DGHUC2|Zoo Med Laborator...|
|B000XY7C7C|All Four Paws, Th...|
|B004UUE26O|Animal Planet PET...|
|B008APML2C|Chuckit Medium Ul...|
|B00ZJN7T8E|1 Half Portion Ja...|
|B00NRZC8LY|Hide-A-Toy Hallow...|
|B001P3NU4E|Virbac C.E.T. Enz...|
|B003TEQ2U6|Jolly Pets Jolly ...|
|B00DJSNF0M|Multipet Lenny th...|
|B00FXVFEQG|Bags on Board Dur...|
|B005ORDWYA|Just One Bite II ...|
|B003E77OG4|  Petmate Sky Kennel|
+----------+--------------------+
only showing top 20 rows



### Customers DataFrame

In [9]:
from pyspark.sql.functions import desc
# Customers DataFrame
customers_df = pet_products_df.groupby("customer_id").agg({"customer_id":"count"})
customers_df.show()



+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   21105356|                 5|
|   29425552|                 2|
|   17041448|                 5|
|   37882821|                 1|
|   42532905|                 1|
|   18759528|                 1|
|   19172324|                 4|
|   51474275|                 2|
|   27911347|                 9|
|   22602819|                 2|
|    5658913|                34|
|   14691760|                 2|
|   13645331|                 1|
|   14139374|                 1|
|   14397122|                 4|
|   10850500|                 1|
|   47233188|                 6|
|   24215961|                 1|
|   16468643|                 1|
|   18630443|                 8|
+-----------+------------------+
only showing top 20 rows



In [10]:
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   22500722|           258|
|   31511151|           236|
|   16053845|           228|
|   25659082|           210|
|   23375624|           200|
|   43856165|           198|
|   51175061|           163|
|   37446839|           160|
|   16540594|           150|
|   16916667|           134|
|   52263513|           134|
|   33315159|           134|
|   18164293|           132|
|   18539854|           132|
|   29676361|           130|
|   39154578|           121|
|   16052325|           120|
|   38352624|           119|
|   50991253|           115|
|   42329785|           115|
+-----------+--------------+
only showing top 20 rows



### Vine DataFrame

In [11]:
# Vine DataFrame
vine_df = pet_products_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1015HJHGZUJVG|          5|            1|          1|   N|
|R1023YMO9XR93S|          2|            0|          0|   N|
|R10267J8PBWEVQ|          5|            0|          0|   N|
|R103326H6Z3CTG|          5|            1|          1|   N|
| R1035J76P280Y|          5|            6|          7|   N|
| R103IP0FCBH8I|          5|            1|          1|   N|
|R103WDQ4IDVIUV|          5|            0|          0|   N|
|R103WQ2GZQLH6T|          5|            0|          0|   N|
|R104K30IR1GWUD|          5|            0|          0|   N|
|R104LLIZQ8O3DW|          5|            0|          0|   N|
|R104OCMU5PF85T|          2|           25|         25|   N|
|R104QASUJNTTNP|          5|            0|          0|   N|
|R1058KF5BARAYI|          5|            0|          0|   N|
|R105G5FW41EYXT|          3|            

# Configuration for RDS instance

In [12]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/pet_products_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

# Write DataFrames to tables to AWS RDS

In [13]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [14]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [15]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [16]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)